Este notebook se encarga de la fase inicial del TFG en la que iremos viendo una versión básica del fetching de noticias con APIs como GNews o newspaper4k, para la aplicación del modelo de IA moral_strength en la que le asignará valores morales a cada noticia por separado.

#### IMPORTS INICIALES

In [70]:
%pip install langchain_openai
%pip install gnews

Note: you may need to restart the kernel to use updated packages.
  Using cached gnews-0.4.2-py3-none-any.whl (18 kB)
  Using cached dnspython-2.8.0-py3-none-any.whl (331 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install lxml_html_clean
%pip install langchain
%pip install load_dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [67]:
import newspaper
import requests
import moralstrength
from moralstrength.moralstrength import estimate_morals

In [79]:
article = newspaper.article("https://www.elespanol.com/mundo/america/eeuu/20251014/trump-amenaza-espana-aranceles-sanchez-no-sube-gasto-defensa-deberian-castigados/1003743969570_0.html")

10/15/2025 05:49:25 PM - get_html_status(): bad status code 403 on URL: https://www.elespanol.com/mundo/america/eeuu/20251014/trump-amenaza-espana-aranceles-sanchez-no-sube-gasto-defensa-deberian-castigados/1003743969570_0.html, html: <!doctype html><html> <head> <title>Voight-Kampff Browser Test</title> <style>#main{margin: auto; width: 100%; padding: 10px;}#content{font-family: sans-serif; max-width: 800px; width: 100%; text-alig


ArticleException: Article `download()` failed with Status code 403 for url None on URL https://www.elespanol.com/mundo/america/eeuu/20251014/trump-amenaza-espana-aranceles-sanchez-no-sube-gasto-defensa-deberian-castigados/1003743969570_0.html

In [58]:
article.meta_lang

'es'

Posible implementación de traducción automática

In [60]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from os import getenv
from dotenv import load_dotenv

load_dotenv(override=True)

template = """Text: {question}
Answer: Translate the text to English in a clear concise way which preserves the same meaning as the original article."""

prompt = PromptTemplate(template=template, input_variables=["question"])

llm = ChatOpenAI(
  api_key=getenv("OPENROUTER_API_KEY"),
  base_url=getenv("OPENROUTER_BASE_URL"),
  model=getenv("MODEL_NAME"),
#   default_headers={
#     "HTTP-Referer": getenv("YOUR_SITE_URL"),
#     "X-Title": getenv("YOUR_SITE_NAME"),
#   }
)

llm_chain = LLMChain(prompt=prompt, llm=llm)

question = article.text

translated_text = llm_chain.run(question)


Ahora hemos conseguido traducir el texto, y podemos pasarlo por moralstrength que funciona con textos en inglés.

In [80]:
translated_text.join("\n")

'\n'

AttributeError: 'list' object has no attribute 'join'

[IMPORTANTE] meterlo dentro de una lista siempre, para que lo detecte como un sólo texto.

In [85]:
result = estimate_morals([translated_text], process=True)
result

/mnt/c/Users/ignacio.escribano_bl/OneDrive/TFG/code/notebooks/.venv/lib/python3.11/site-packages/spacy/pipeline/lemmatizer.py:188: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


,care,fairness,loyalty,authority,purity
0,5.57013,7.494048,8.697321,6.152083,NaN


Pasémoslo a una función para hacerlo más fácil

In [ ]:
class LLMChat(object):
    """
    Class to handle LLML operations via langchain
    """

    def __init__(self):
        load_dotenv(override=True)

        llm = ChatOpenAI(
            api_key=getenv("OPENROUTER_API_KEY"),
            base_url=getenv("OPENROUTER_BASE_URL"),
            model=getenv("MODEL_NAME"),
        )

        self._llm = llm

        template = """Text: {question}
        Answer to the question or task in raw text without markdown, in a clear and concise way."""

        prompt = PromptTemplate(template=template, input_variables=["question"])
        self._llmchat = LLMChain(prompt=prompt, llm=llm)



  
    def translate_text_llm(self, text: str) -> str:
        question = f"Translate the following text to English in a way that in conserves \
            the original meaning and writing style: {text}"
        try:
            result = self._llmchat.run(question=question)
            return result
        except Exception as e:
            print(f"There was an error invoking the LLM: {e}")
            return e
        
        

In [66]:
llmchat = LLMChat()

translated_article = llmchat.translate_text_llm(text=article.text)


In [72]:
translated_article

'U.S. President Donald Trump remains determined that Spain increase its defense spending to 5% of GDP. The Republican has again threatened this Tuesday to impose tariffs on Spain if Pedro Sánchez’s government refuses to raise it.\n\nDuring a press conference at the White House alongside Argentine President Javier Milei, the Republican stated he is “very unhappy” with Spain, calling its refusal to increase spending “disrespectful” toward other NATO allies.\n\n“It’s a major disrespect to NATO. In fact, I was considering imposing a commercial penalty on them through tariffs for what they did. And I think I could do it—it’s incredibly disrespectful. Spain is the only one among all NATO countries to say that, and I believe it should be punished for it,” he warned.\n\nGovernment sources downplayed this threat of commercial retaliation, noting that the bilateral relationship was evident Monday through the greeting between Trump and Sánchez in Egypt.\n\nThis threat to impose tariffs is not new

In [73]:
from gnews import GNews
import newspaper

# 1. Search news
google_news = GNews()
results = google_news.get_news('economía')

llmchat = LLMChat()

# 2. Extract full article body
for item in results:
    url = item['url']
    article = newspaper.article(url)


    if article.meta_lang != "en":
        article = llmchat.translate_text_llm(text=article.text)
    
    print("Title:", article.title)
    print("Text:", article.text[:300], "...")
    print(estimate_morals(article.text))


Title: Google News
Text:  ...
Empty DataFrame
Columns: [care, fairness, loyalty, authority, purity]
Index: []
Title: Google News
Text:  ...
Empty DataFrame
Columns: [care, fairness, loyalty, authority, purity]
Index: []
Title: Google News
Text:  ...
Empty DataFrame
Columns: [care, fairness, loyalty, authority, purity]
Index: []
Title: Google News
Text:  ...
Empty DataFrame
Columns: [care, fairness, loyalty, authority, purity]
Index: []
Title: Google News
Text:  ...
Empty DataFrame
Columns: [care, fairness, loyalty, authority, purity]
Index: []
Title: Google News
Text:  ...
Empty DataFrame
Columns: [care, fairness, loyalty, authority, purity]
Index: []
Title: Google News
Text:  ...
Empty DataFrame
Columns: [care, fairness, loyalty, authority, purity]
Index: []
Title: Google News
Text:  ...
Empty DataFrame
Columns: [care, fairness, loyalty, authority, purity]
Index: []
Title: Google News
Text:  ...
Empty DataFrame
Columns: [care, fairness, loyalty, authority, purity]
Index: []
Title: Goo

HABRÍA QUE PASAR TODO EL TEXTO JUNTO, QUE NO LO SEPARE AUTOMÁTICAMENTE

In [78]:
import newspaper


paper = newspaper.build("https://www.elpais.com/", memoize_articles=False)

print(f"Found {len(paper.articles)} articles.")

llmchat = LLMChat()

# 3️⃣ Process each article
for i, art in enumerate(paper.articles[:10], 1):  # limit to 10 for demo
    try:
        art.download()
        art.parse()
    except Exception as e:
        print(f"❌ Error downloading article {i}: {e}")
        continue

    text = art.text.strip()
    if not text:
        continue

    try:
        lang = art.meta_lang
    except Exception:
        lang = "unknown"

    if lang != "en":
        print(f"Article {i}: Detected {lang}. Translating to English...")
        translated_text = llmchat.translate_text_llm(text=text)
    else:
        translated_text = text

    print(f"\n📰 {i}. {art.title}")
    print(f"URL: {art.url}\n")
    print(translated_text[:400], "...\n")

    # Run your custom analysis
    print("Moral estimation:", estimate_morals(translated_text, process=True))
    print("-" * 80)


/mnt/c/Users/ignacio.escribano_bl/OneDrive/TFG/code/notebooks/.venv/lib/python3.11/site-packages/newspaper/source.py:260: FutureWarning: Truth-testing of elements was a source of confusion and will always return True in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  if feed.doc:


Found 3 articles.
Article 1: Detected es. Translating to English...


10/15/2025 05:28:17 PM - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"



📰 1. El Estado Mayor ruso apunta a la conquista de Odesa como objetivo de la invasión de Ucrania
URL: https://elpais.com/internacional/2025-09-02/el-estado-mayor-ruso-apunta-a-la-conquista-de-odesa-como-objetivo-de-la-invasion-de-ucrania.html

Russian Chief of the General Staff Valery Gerasimov, architect of the 2022 “blitzkrieg” offensive, reviewed this past weekend the results of his summer military campaign in the fourth year of Russia’s invasion of Ukraine. In the background, discreetly displayed, hung a map that has drawn attention in Russia. It showed not only the five Ukrainian regions that Russian President Vladimir Putin has fo ...



/mnt/c/Users/ignacio.escribano_bl/OneDrive/TFG/code/notebooks/.venv/lib/python3.11/site-packages/spacy/pipeline/lemmatizer.py:188: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


...


📰 2. Zelenski fulmina a uno de los últimos referentes de la Ucrania próxima a Rusia
URL: https://elpais.com/internacional/2025-10-15/zelenski-fulmina-a-uno-de-los-ultimos-referentes-de-la-ucrania-proxima-a-rusia.html

Gennadi Trukhanov’s political career has always been in the eye of the storm. The mayor of Odesa, Ukraine’s third-largest city, has withstood 11 years under accusations of pro-Moscow sympathies. Voters elected him three times while the Security Service of Ukraine (SSU) sought evidence that the mayor held Russian citizenship. The saga ended Tuesday, when President Volodymyr Zelensky swiftly signed  ...



/mnt/c/Users/ignacio.escribano_bl/OneDrive/TFG/code/notebooks/.venv/lib/python3.11/site-packages/spacy/pipeline/lemmatizer.py:188: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


Moral estimation:       care  fairness  loyalty  authority  purity
0      NaN       NaN      NaN        NaN     NaN
1      NaN       NaN      NaN        NaN     NaN
2      NaN       NaN      NaN        NaN     NaN
3      NaN       NaN      NaN        NaN     NaN
4      NaN       NaN      NaN        NaN     NaN
...    ...       ...      ...        ...     ...
5942   NaN       NaN      NaN        NaN     NaN
5943   NaN       NaN      NaN        NaN     NaN
5944   NaN       NaN      NaN        NaN     NaN
5945   NaN       NaN      NaN        NaN     NaN
5946   NaN       NaN      NaN        NaN     NaN

[5947 rows x 5 columns]
--------------------------------------------------------------------------------


10/15/2025 05:32:05 PM - get_html_status(): bad status code 404 on URL: https://cadenaser.com/nacional/2025/10/13/un-magistrado-del-tribunal-constitucional-confirma-una-de-las-leyendas-sobre-el-palco-del-santiago-bernabeu-cadena-ser/ , html: <!DOCTYPE html><html lang="es"><head><meta charSet="utf-8"/><meta name="viewport" content="width=device-width, initial-scale=1"/><meta name="format-detection" content="address=no;email=no,telephone:no


❌ Error downloading article 3: Article `download()` failed with Status code 404 for url None on URL https://cadenaser.com/nacional/2025/10/13/un-magistrado-del-tribunal-constitucional-confirma-una-de-las-leyendas-sobre-el-palco-del-santiago-bernabeu-cadena-ser/ 


In [1]:
import feedparser

In [ ]:
d = feedparser.parse('https://rss.elpais.com/rss/elpais/portada.xml')

print(f"Feed: {d.feed}")
print(f"Feed structure: {d.entries[0]}")